<a href="https://colab.research.google.com/github/mglavan07/HYDAC_Capacity_Forecasting_Models/blob/main/DataSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# define a function to gather all data in a directory into one large dataframe
def directory_to_dataframe():
    try:
        # required module
        import pandas as pd

        # list of dataframes for each excel file
        dfs = []

        # collect files
        from google.colab import files
        uploaded = files.upload()

        if len(uploaded) == 0:
            raise ValueError("Error: select at least one file")
            return pd.DataFrame()

        # import all data from the passed directory
        for filename in uploaded:
            if filename.lower().endswith(".xlsx"):

                # convert into a dataframe
                local_df = pd.read_excel(filename)
                dfs.append(local_df)

        # combine all data frames
        if not dfs:
            raise ValueError(f"No .xlsx files passed")
        df = pd.concat(dfs, ignore_index = True)

        # return the dataframe
        return df

    # handle errors
    except ModuleNotFoundError:
        print("Error: pandas is not installed.")
        return pd.DataFrame()
    except ValueError as ve:
        print(f"{ve}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

In [2]:
# define a function to separate the relevant columns of data
def clean_dataframe(df):
    # FILTERED COLS:
    # DATE:
    # MATERIAL:
    # NET SALES
    # COST
    # INVOICED (QTY):
    try:
        # import pandas
        import pandas as pd

        # locate columns of interest
        date_col = df.columns.get_loc("Date")
        material_col = df.columns.get_loc("Material")
        sales_col = df.columns.get_loc("NET SALES")
        cost_col = df.columns.get_loc("Cost")
        qty_col = df.columns.get_loc("Invoiced Quantity")

        # slice the dataframe
        return df.iloc[:, [date_col, material_col, sales_col, cost_col, qty_col]]

    # flag errors
    except KeyError:
        print(f"Error: not all columns located")
        return pd.DataFrame()
    except ModuleNotFoundError:
        print("Error: pandas is not installed.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

In [3]:
# define a function that condenses info under each material number
def condense_materials(clean_df, sort = True):
  try:
    # import pandas
    import pandas as pd

    # Group by 'Material' and sum 'Invoiced Quantity' and 'NET SALES'
    condensed_df = clean_df.groupby("Material", as_index=False)[["Invoiced Quantity", "NET SALES", "Cost"]].agg(lambda x: x.abs().sum())

    # Rename columns for clarity
    condensed_df = condensed_df.rename(columns={
          "Invoiced Quantity": "Total Quantity",
          "NET SALES": "Total Revenue",
          "Cost": "Total Cost"
      })

    # optional: sort
    if (sort):
        condensed_df = condensed_df.sort_values(by='Total Revenue', ascending=False)

    # return the dataframe
    return condensed_df

  # flag errors
  except KeyError:
      print(f"Error: not all columns located")
      return pd.DataFrame()
  except ModuleNotFoundError:
      print("Error: pandas is not installed.")
      return pd.DataFrame()
  except Exception as e:
      print(f"An unexpected error occurred: {e}")
      return pd.DataFrame()

In [4]:
# define a function to go from start to finish in one call
def directory_to_precluster(raw, sort_rev = True):

   # filter the relevant rows
   clean = clean_dataframe(raw)

   # group and sum rows of the same material
   condensed = condense_materials(clean, sort = sort_rev)

   # return final product
   return condensed

**Column Information:**

**Total Quanity** - How many of the material we have historically produced

**Total Revenue** - How much of the revenue generated by the plant is attributed to the manufacturing of the material

**Total Cost** - How much plant infrastructure has been dedicated to the production of the good over time

In [5]:
# test the functions
if __name__ == "__main__":
  data = directory_to_dataframe()
  print(directory_to_precluster(data, sort_rev=True))


Saving 16_06_20_OK_21stCenturySales.XLSX to 16_06_20_OK_21stCenturySales.XLSX
Saving 16_06_30_OK_21stCenturySales.XLSX to 16_06_30_OK_21stCenturySales.XLSX
     Material  Total Quantity  Total Revenue  Total Cost
235   4225610             843     2553007.50  1453272.37
203   2958512             231      428114.05   364004.56
8     2595008             112      325401.00   229879.05
30    2597969             104      297878.39   219902.78
127   2952905              40      225607.46   126930.09
..        ...             ...            ...         ...
9     2595281               1         459.00      355.08
10    2596278               1         440.00      309.91
65    2598663               1         427.48      338.26
14    2596888               2         423.38      258.26
194   2957508               1         160.00      196.21

[288 rows x 4 columns]
